In [16]:
import iris
import numpy
from pyproj import Proj
from shapely.geometry import shape

## Read the data

The idea is to use iris so that the `cube.coord('latitude').guess_bounds()` method can be used if the bounds aren't supplied. (This can be done for latitude, longitude or level). The values held in an iris cube are `numpy.ma.core.MaskedArray`.

In [17]:
infile = "/Users/irv033/Downloads/data/thetao_Omon_CSIRO-Mk3-6-0_piControl_r1i1p1_000101-001012.nc"

In [40]:
def lat_subset(cell):
    return -80 < cell < -55

def lon_subset(cell):
    return 30 < cell < 70

lat_constraint = iris.Constraint(latitude=lat_subset)
lon_constraint = iris.Constraint(longitude=lon_subset)

In [41]:
with iris.FUTURE.context(cell_datetime_objects=True):
    cube = iris.load_cube(infile, 'sea_water_potential_temperature' & lat_constraint & lon_constraint)

In [42]:
print cube

sea_water_potential_temperature / (K) (time: 120; depth: 31; latitude: 26; longitude: 20)
     Dimension coordinates:
          time                             x           -             -              -
          depth                            -           x             -              -
          latitude                         -           -             x              -
          longitude                        -           -             -              x
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@qld.gov.au) & Leon Rotstayn...
          creation_date: 2011-05-11T07:04:01Z
          experiment: pre-industrial co

In [43]:
lat_bounds = cube.coord('latitude').bounds
lon_bounds = cube.coord('longitude').bounds
lev_bounds = cube.coord('depth').bounds

print lat_bounds.shape
print lon_bounds.shape
print lev_bounds.shape

print type(lat_bounds)

(26, 2)
(20, 2)
(31, 2)
<type 'numpy.ndarray'>


In [44]:
lat_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lat_bounds)
lon_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lon_bounds)
lev_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lev_bounds)

In [45]:
print lat_diffs.shape
print lat_diffs[3]
print lat_bounds[3,:]

(26,)
0.933143615723
[-76.48403168 -75.55088806]


## Perform the integration

Rather than use [`scipy.integrate.simps`](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.integrate.simps.html) or something similar, I think it's best to use the level bounds provided in the dataset to essentially define and calculate the area of a series of rectangles.

#### Single point

In [46]:
test_point = cube.data[1, :, 1, 1]
print test_point
print type(test_point)

[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- --]
<class 'numpy.ma.core.MaskedArray'>


In [47]:
areas = test_point * lev_diffs
print areas
print areas.sum()

[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- --]
--


#### Whole array

In [48]:
def simple_integration(vector):
    """Perform integration for a single vector (of length of level dimension)"""
    
    areas = vector * lev_diffs
    
    return areas.sum()

In [49]:
OHC_per_m2 = numpy.ma.apply_along_axis(simple_integration, 1, cube.data)

In [50]:
print OHC_per_m2.shape
print type(OHC_per_m2)
print OHC_per_m2[1,1,1]

(120, 26, 20)
<class 'numpy.ma.core.MaskedArray'>
--


## Multiply by the area

Following [this](http://stackoverflow.com/questions/4681737/how-to-calculate-the-area-of-a-polygon-on-the-earths-surface-using-python) Stack Overflow response...

In [51]:
co = {"type": "Polygon", "coordinates": [
    [(-102.05, 41.0),
     (-102.05, 37.0),
     (-109.05, 37.0),
     (-109.05, 41.0)]]}
lon, lat = zip(*co['coordinates'][0])
pa = Proj("+proj=aea +lat_1=37.0 +lat_2=41.0 +lat_0=39.0 +lon_0=-106.55")

In [52]:
x, y = pa(lon, lat)
cop = {"type": "Polygon", "coordinates": [zip(x, y)]}
shape(cop).area 

268952044107.43454